# textCNN

## 1.实际训练阶段

### 1.1引入模块库

In [1]:
from MyModule import Net
import cupy as cp
import numpy
import matplotlib as plt

Enjoy NPL MachineLearning


### 1.2导入训练集
格式为 batch h= w= channels=

In [2]:
x_1 = numpy.load('Data/negative.npy',allow_pickle=True)
x_2 = numpy.load('Data/positive.npy',allow_pickle=True)
y_1 = numpy.repeat([[0,1]],109,axis=0)
y_2 = numpy.repeat([[1,0]],202,axis=0)
print(f"y_1 shape:{y_1.shape} y_2 shape: {y_2.shape}")
origin_X = numpy.concatenate((x_1,x_2))
origin_Y = numpy.concatenate((y_1,y_2))
# 打乱数据集
per = numpy.random.permutation(origin_X.shape[0])#打乱后的行号
train_x = origin_X[per, :, :]#获取打乱后的训练数据
train_y = origin_Y[per]
print(train_x.shape)
print(train_y.shape)
train_x = train_x.reshape((311,20,100,1))
train_x = cp.asarray(train_x)
train_y = cp.asarray(train_y)
print(train_x.shape,train_y.shape)
# print(train_x[0])

y_1 shape:(109, 2) y_2 shape: (202, 2)
(311, 20, 100)
(311, 2)
(311, 20, 100, 1) (311, 2)


### 1.3定义网络结构

In [3]:
# 卷积层
conv1 = Net.Conv(kernal=(3,100,1,10),stride=1)
conv2 = Net.Conv(kernal=(4,100,1,10),stride=1)
conv3 = Net.Conv(kernal=(5,100,1,10),stride=1)

# 激活函数Relu
relu1 = Net.Relu()
relu2 = Net.Relu()
relu3 = Net.Relu()

# 池化层
max_pooling1 = Net.Pool(ksize=2)
max_pooling2 = Net.Pool(ksize=2)
max_pooling3 = Net.Pool(ksize=2)

# Flatten 层
flatten1 = Net.Flatten()
flatten2 = Net.Flatten()
flatten3 = Net.Flatten()

# Concat融合
concat1 = Net.Concat()

# 全连接层
fc1 = Net.Fc( 250,2)

# softmax
softmax = Net.Softmax()

### 定义前向传播函数

In [4]:
def Forward(X,Y):
    # 卷积
    forward1 = conv1.forward(X)
    forward2 = conv2.forward(X)
    forward3 = conv3.forward(X)
    # print(forward1.shape)
    # relu
    forward1 = relu1.forward(forward1)
    forward2 = relu2.forward(forward2)
    forward3 = relu3.forward(forward3)
    # print(forward1.shape)
    # 池化
    forward1 = max_pooling1.forward(forward1)
    forward2 = max_pooling2.forward(forward2)
    forward3 = max_pooling3.forward(forward3)
    # print(forward1.shape)
    # flatten
    forward1 = flatten1.forward(forward1)
    forward2 = flatten2.forward(forward2)
    forward3 = flatten3.forward(forward3)
    # print(forward1.shape)
    # print(forward2.shape)
    # print(forward3.shape)
    # concate
    forward = concat1.forward(forward1,forward2,forward3)
    # print(forward.shape)
    # 全连接
    forward = fc1.forward(forward)
    # print(forward.shape)
    # softmax
    loss,delta = softmax.calcuteLoss(forward,Y)
    predict = softmax.predict(forward)
    right = 0
    # print(predict.shape)
    for i in range(X.shape[0]):
        if cp.argmax(predict[i])==cp.argmax(Y[i]) :
            right = right + 1
    return loss,delta,right/X.shape[0]
# print(f"softmax_layer:{delta.shape}" )
# print(f"loss:{loss} delta:{delta}")

### 定义反向传播函数

In [5]:
def Backward(delta,lr):
    # 全连接
    backward = fc1.backward(delta,lr)
    # print(f"fc:{backward.shape}")
    # concate
    backward1,backward2,backward3 =  concat1.backward(backward)
    # print(f"concate:{backward1.shape} {backward2.shape} {backward3.shape}")
    # flatten
    backward1 = flatten1.backward(backward1)
    backward2 = flatten2.backward(backward2)
    backward3 = flatten3.backward(backward3)
    # print(f"flatten1:{backward1.shape} flatten2:{backward2.shape} flatten3:{backward3.shape}")
    # 池化
    backward1 = max_pooling1.backward(backward1)
    backward2 = max_pooling2.backward(backward2)
    backward3 = max_pooling3.backward(backward3)
    # print(f"pool1:{backward1.shape} pool2:{backward2.shape} pool3:{backward3.shape}")
    # relu
    backward1 = relu1.backward(backward1)
    backward2 = relu2.backward(backward2)
    backward3 = relu3.backward(backward3)
    # 卷积
    backward1 = conv1.backward(backward1,lr)
    backward2 = conv2.backward(backward2,lr)
    backward3 = conv3.backward(backward3,lr)
    # print(f"反向传播完成")

#### 1.4.3训练结果和损失展示

In [6]:
learning_rate = 0.01
batch_size = 10
epoches =10
img_x = []
loss_y = []
right_y = []
for epoch in range(epoches+1):
    for i in range(0,211,batch_size): 
        X = train_x[i:i+batch_size]
        Y = train_y[i:i+batch_size]
        loss,delta,right_rate= Forward(X,Y)
        Backward(delta,learning_rate)
        # print("==============================================")
        print(f"epoch-{epoch}/{epoches}-{i}/{311} loss:{loss} right_rate :{right_rate} ")
        # print(Y)
    learning_rate *= 0.95**(epoch+1)
    

epoch-0/10-0/311 loss:6.268633042357708 right_rate :0.2 
epoch-0/10-10/311 loss:1.6832112605418672 right_rate :0.6 
epoch-0/10-20/311 loss:1.883265050246677 right_rate :0.3 
epoch-0/10-30/311 loss:0.6742724076953441 right_rate :0.7 
epoch-0/10-40/311 loss:0.5518002881494237 right_rate :0.8 
epoch-0/10-50/311 loss:0.920458684572964 right_rate :0.5 
epoch-0/10-60/311 loss:0.6964953041876207 right_rate :0.6 
epoch-0/10-70/311 loss:0.585530874225778 right_rate :0.8 
epoch-0/10-80/311 loss:0.8913459850626129 right_rate :0.5 
epoch-0/10-90/311 loss:0.6694711835326967 right_rate :0.9 
epoch-0/10-100/311 loss:0.6698114148739105 right_rate :0.7 
epoch-0/10-110/311 loss:0.6873196847898749 right_rate :0.6 
epoch-0/10-120/311 loss:0.6840415756744237 right_rate :0.5 
epoch-0/10-130/311 loss:0.6646786159910705 right_rate :0.6 
epoch-0/10-140/311 loss:0.8333917912166111 right_rate :0.5 
epoch-0/10-150/311 loss:0.6666120417151088 right_rate :0.6 
epoch-0/10-160/311 loss:0.6825524546987741 right_rate :

#### 1.4.4 预测结果展示

In [7]:

batch_size = 1
epoches =1
img_x = []
loss_y = []
right_y = []
count = 0
for i in range(211,311,batch_size): 
    X = train_x[i:i+batch_size]
    Y = train_y[i:i+batch_size]
    loss,delta,right_rate= Forward(X,Y)
    if right_rate==1 : count= count+1
    # print(right_rate)
print(f"right_rate: {(count/(311-211))*100}%")


right_rate: 61.0%
